In [1]:
from pyspark import SparkContext, SparkConf

# local - 本地运行
conf = SparkConf().setMaster("local").setAppName("myApp")
sc = SparkContext(conf=conf)

In [2]:
# 创建时指定分区数
# 1个分区
rdd_demo = sc.parallelize([1, 2, 3, 4, 5, 6])

In [3]:
# 查看rdd分区数
rdd_demo.getNumPartitions()

1

In [15]:
# 2个分区
rdd_demo = sc.parallelize([1, 2, 3, 4, 5, 6], 2)

In [5]:
# glom查看分区元素
rdd_demo.glom().collect()

[[1, 2, 3], [4, 5, 6]]

In [8]:
rdd1 = sc.parallelize([1, 2, 3, 4, 5, 6], 4)

In [11]:
rdd1.glom().collect()

[[1], [2, 3], [4], [5, 6]]

In [16]:
# mapPartitions 对分区进行map
def func1(iterator):
    yield sum(iterator)  # 只能用迭代器函数

In [17]:
rdd_demo.mapPartitions(func1).collect()

[6, 15]

In [36]:
# mapPartitionsWithIndex - 对分区map时传入分区id
def func2(index, iteratior):
    lists = list(iteratior)
    print(index, '=>', lists)
    yield (index, sum(lists))

In [37]:
rdd_demo.mapPartitionsWithIndex(func2).collect()

[(0, 6), (1, 15)]

In [55]:
# foreachPartition(func) - 对分区的每一个元素进行func处理
def func3(iter):
    print('sum is', sum(iter))

In [56]:
rdd_demo.foreachPartition(func3)

In [40]:
# 练习
# 目标：计算出每个分区的a、b的和
# [
# [0, ('a', 140), ('b', 340)],
# [1, ('a', 986), ('b', 462)]
# ]
# 使用mapPartitionsWithIndex计算每个分区，每个键的总和
rdd = sc.parallelize([("a", list(range(10))), ("a", list(range(5, 15))),
                      ("b", list(range(10, 20))), ("b", list(range(15, 25))),
                      ("a", list(range(25, 32))), ("a", list(range(20, 30))),
                      ("b", list(range(63, 70))), ("a", list(range(50, 60)))],
                     2)

In [41]:
rdd.glom().collect()

[[('a', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
  ('a', [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]),
  ('b', [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
  ('b', [15, 16, 17, 18, 19, 20, 21, 22, 23, 24])],
 [('a', [25, 26, 27, 28, 29, 30, 31]),
  ('a', [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
  ('b', [63, 64, 65, 66, 67, 68, 69]),
  ('a', [50, 51, 52, 53, 54, 55, 56, 57, 58, 59])]]

In [42]:
def func4(index, iter):
    lists = list(iter)
    yield (index, lists)

In [43]:
rdd.mapPartitionsWithIndex(func4).collect()

[(0,
  [('a', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
   ('a', [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]),
   ('b', [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
   ('b', [15, 16, 17, 18, 19, 20, 21, 22, 23, 24])]),
 (1,
  [('a', [25, 26, 27, 28, 29, 30, 31]),
   ('a', [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
   ('b', [63, 64, 65, 66, 67, 68, 69]),
   ('a', [50, 51, 52, 53, 54, 55, 56, 57, 58, 59])])]

In [47]:
def func5(index, iter):
    lists = list(iter)
    print(f'{index} => {lists}')
    res = {'a': 0, 'b': 0}
    for i in lists:
        res[i[0]] += sum(i[1])
        print(i[0], '=>', sum(i[1]))
    yield [index, ('a', res['a']), ('b', res['b'])]

In [48]:
rdd.mapPartitionsWithIndex(func5).collect()

[[0, ('a', 140), ('b', 340)], [1, ('a', 986), ('b', 462)]]

In [57]:
# 分区下的combineByKey
new_pair_rdd = sc.parallelize([("a", [1, 2, 3]), ("b", [4, 5, 6]),
                               ("a", [10, 20, 30]), ("b", [40, 50, 60]),
                               ("a", [7, 8, 9]), ("b", [10, 11, 12]),
                               ("a", [70, 80, 90]), ("b", [100, 110, 120])], 2)

In [58]:
def create_list(first_item):
    print("first_item=", first_item)
    return [sum(first_item)]


# 将后面遇到的相同的key元素添加到对应的列表
def append(last_result, current_item):
    print("args:", last_result, current_item)
    last_result.append(sum(current_item))
    return last_result


# 如果只有一个分区这个函数不会被调用
def extend(last_partitioner, current_partinioner):
    print("partitioner:", last_partitioner, current_partinioner)
    last_partitioner.extend(current_partinioner)
    return last_partitioner

In [60]:
new_rdd = new_pair_rdd.combineByKey(create_list, append, extend)

In [61]:
new_rdd.collect()

[('b', [15, 150, 33, 330]), ('a', [6, 60, 24, 240])]